In [11]:
import numpy as np  
import pandas as pd 
import matplotlib.pyplot as plt  
import seaborn as sns 

In [12]:
google_analytic = pd.read_csv("data/google_analytic.csv")

In [13]:
google_analytic['ID'] = google_analytic['Page location'].str.extract(r'utm_id=(\d+)').fillna(-1).astype(int)


In [14]:
sam_us = pd.read_csv("data/October-Target-SAM-List-US-All.csv")
sam_canada = pd.read_csv("data/October-Target-SAM-List-CAN-All.csv")

sam_list = pd.concat([sam_us, sam_canada], ignore_index=True)
sam_list['Snowball Map'] = sam_list['Snowball Map'].astype(int)


In [15]:
merged_df = pd.merge(google_analytic, sam_list, left_on='ID', right_on='Snowball Map', how='left')

merged_df['ID'] = merged_df['ID'].astype(str).str.zfill(5)


merged_df['Date'] = pd.to_datetime(merged_df['Date'], format='%Y%m%d')

# If you want to keep the date in the 'YYYY-MM-DD' format, you can format it accordingly
merged_df['Date'] = merged_df['Date'].dt.strftime('%Y-%m-%d')


# Count the number of occurrences of each ID
id_counts = merged_df['ID'].value_counts()

# Create the 'visit_again' column based on the ID counts
merged_df['visit_again'] = merged_df['ID'].map(id_counts) > 1

# Fill any NaN values with False (for IDs that only appear once)
merged_df['visit_again'] = merged_df['visit_again'].fillna(False)

merged_df2 = merged_df.drop_duplicates(subset='Page location', keep='last').copy()


In [16]:
merged_df2.to_csv("final_data.csv", index=False)
